In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras import models, layers




In [ ]:
# Set up paths to audio files and genre labels
AUDIO_PATH = 'audio'
CSV_PATH = 'data.csv'


In [ ]:
# Load audio files and extract features using librosa
def extract_features(file_path):
    audio_data, _ = librosa.load(file_path, sr=22050, mono=True, duration=30)
    mfccs = librosa.feature.mfcc(y=audio_data, sr=22050, n_mfcc=20)
    chroma_stft = librosa.feature.chroma_stft(y=audio_data, sr=22050)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio_data, sr=22050)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio_data, sr=22050)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio_data, sr=22050)
    features = np.concatenate((np.mean(mfccs, axis=1), np.mean(chroma_stft, axis=1), np.mean(spectral_centroid), np.mean(spectral_bandwidth), np.mean(spectral_rolloff)))
    return features

In [ ]:
# Load data from CSV file and extract features
data = pd.read_csv(CSV_PATH)
features = []
labels = []
for index, row in data.iterrows():
    file_path = os.path.join(AUDIO_PATH, row['filename'])
    genre = row['label']
    features.append(extract_features(file_path))
    labels.append(genre)

In [ ]:
# Encode genre labels and scale features
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
scaler = StandardScaler()
features = scaler.fit_transform(np.array(features, dtype=float))


In [ ]:

# Split data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2)

In [ ]:
# Build a neural network for music genre classification
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(train_features.shape[1],)))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training set and evaluate it on the testing set
history = model.fit(train_features, train_labels, epochs=50, batch_size=128, validation_data=(test_features, test_labels))


In [ ]:
# Plot the training and testing accuracy and loss curves
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Testing Accuracy')
plt.title('Training and Testing Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()



In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Testing Loss')
plt.title('Training and Testing Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()